In [ ]:
import numpy as np
import scipy

## UW Directed Graph Embedding Algorithm

![UW Directed Embedding Algorithm](../images/UW_directed_embedding_alg.png)

In [ ]:
"""
UW directed embedding algorithm
input: W affinity matrix, m embedding dimension
"""
def directed_embedding(W, m):
    # steps 1-6 (estimate coordinates according to diffusion map)
    # step 1
    S = (W + np.transpose(W))/2 # symmetric affinity matrix

    # step 2
    q = [np.sum(row) for row in S]
    Q = np.diag(q) # degree matrix

    # step 3
    Qinv = np.linalg.inv(Q)
    V = Qinv@S@Qinv # anisotropic normalized affinity matrix

    # step 4
    q1 = [np.sum(row) for row in V]
    Q1 = np.diag(q) # normalized degree matrix
    Q1inv = np.linalg.inv(Q1) # 
    
    # step 5
    Hss = Q1inv@V # diffusion matrix (dividing affinity matrix by row sums)

    # step 6
    eig_vals, eig_vecs = np.linalg.eig(Hss)
    idx = eig_vals.argsort()[::-1]   
    eig_vals = eig_vals[idx]
    eig_vecs = eig_vecs[:,idx]

    phi = eig_vecs[:(m+1)] # first (m+1) right eigenvectors
    coords = eig_vecs[:,1:(m+1)] # first m significant right eigenvectors (coordinates of points in embedding)
    A = np.diag(eig_vals) # diagonal matrix of (m+1) significant eigenvalues


    # step 7-8: estimate the density
    evals, evecs = scipy.linalg.eig(Hss, left = True, right = False)
    pi = evecs[np.where(evals == 1)] # left eigenvector of Hss with eigenvalue 1
    density = pi/sum(pi) # normalize

    # step 9-13: estimate vector field r
    # step 9
    p = [np.sum(row) for row in W]
    P = np.diag(p)

    # step 10
    Pinv = np.linalg.inv(P)
    T = Pinv@W@Pinv
    p1 = [np.sum(row) for row in T]
    P1 = np.diag(p1)

    # step 11
    P1inv = np.linalg.inv(P1)
    Haa = P1@T
    R = ((Haa - Hss)@phi)/2
    fields = R[1:] # vector field components in the direction of the corresponding coordinates of the embedding

    return coords, density, fields

In [ ]:
# Testing Block
S = [[1,0,0,0],
     [1,1,1,0],
     [1,3,0,0],
     [1,2,0,4]]
q = [np.sum(row) for row in S]
Q = np.diag(q)
# print(q)
# print(Q)
# print(Q@S@Q)

eig_vals, eig_vecs = np.linalg.eig(S)
print(eig_vals)
print(eig_vecs)

print("----")

idx = eig_vals.argsort()[::-1]   
eig_vals = eig_vals[idx]
eig_vecs = eig_vecs[:,idx]
phi = eig_vecs[:,:3]
print(eig_vals)
print(phi)

S2 = [[1,0,0],
      [1,0,0],
      [1,0,0]]

S22 = S2 + np.transpose(S2)
# print(S22)


[ 4.          2.30277564 -1.30277564  1.        ]
[[ 0.          0.          0.          0.63799308]
 [ 0.          0.49471995 -0.39390152 -0.42532872]
 [ 0.          0.64450909  0.90706684 -0.63799308]
 [ 1.         -0.5829753   0.14856428  0.07088812]]
----
[ 4.          2.30277564  1.         -1.30277564]
[[ 0.          0.          0.63799308]
 [ 0.          0.49471995 -0.42532872]
 [ 0.          0.64450909 -0.63799308]
 [ 1.         -0.5829753   0.07088812]]
